In [1]:
%load_ext autoreload
%autoreload 2 
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import os
import pandas as pd
import numpy as np 
import pickle as pk
import glob
from fbprophet import Prophet
import sys
sys.path.append('../')
from wiki import utils 
import multiprocessing as mp
total_proc = None
from tqdm import tqdm
from IPython.display import clear_output

In [3]:
PROPHET_PATH = '../data/prophet/'

In [4]:
glob.glob(PROPHET_PATH+'*')

['../data/prophet/ds.f',
 '../data/prophet/page_index.f',
 '../data/prophet/pagedf.f',
 '../data/prophet/v0']

In [5]:
pagedf = pd.read_feather(PROPHET_PATH+'pagedf.f')
ds = pd.read_feather(PROPHET_PATH+'ds.f')

## Version X
Should set version directory name in next cell. Should describe version specifics (outliers, holidays, validation period). How will the validation period be communcated to the SMAPE scoring stuff?

In [6]:
# should break if the dir already exists - avoids accidental overwriting
version = 'v0'
os.makedirs(PROPHET_PATH+version)
PROPHET_PATH = PROPHET_PATH+version+'/'

FileExistsError: [Errno 17] File exists: '../data/prophet/v0'

In [7]:
pagedf = pagedf.loc[:,:'10']

In [9]:
def process_page(page):
    df = ds.join(pagedf[page])
    df.columns = ['ds','y']
    # note this is doing validation on last 60 days
    # should also consider doing validation on the time period we are forcasting
    traindf = df.iloc[:-60]
    # do outlier removal here
    #traindf.loc[traindf.y > traindf.y.quantile(.95), ['y']] = None
    m = Prophet(yearly_seasonality=True)
    m.fit(traindf)
    forecast = m.predict(ds)
    forecast = forecast.join(df.y)
    forecast.to_feather(PROPHET_PATH+page+'df.f')
    with open(PROPHET_PATH+page+'m.pk', 'wb') as file:
        pk.dump(m,file)

In [10]:
mp.cpu_count()

4

In [11]:
with utils.clock():
    mp_pool = mp.Pool()
    with tqdm(total=pagedf.shape[1]) as pbar:
        for i, _ in tqdm(enumerate(mp_pool.imap_unordered(process_page, pagedf))):
            clear_output()
            pbar.update()           
    pbar.close()
    mp_pool.close()
    mp_pool.join()


100%|██████████| 11/11 [00:06<00:00,  1.65it/s]

Elapsed time 6.752970933914185 seconds
